In [11]:
import autogen
# from autogen import ConversableAgent
# from autogen import initiate_chats
from openai import OpenAI
from dotenv import load_dotenv
import os
import pprint


load_dotenv('.secrets')
openAI_api_key = os.getenv("OPENAI_API_KEY")
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")


openAI_config = {
    "config_list": [
        {
            "model": "gpt-3.5-turbo",
            "api_key": openAI_api_key,
        }
    ],
    "timeout": 120  
}

llama_config = {
    "config_list": [
        {
            "model": "meta-llama-3.1-8b-instruct",
            "base_url": "http://192.168.1.170:1234/v1",
            "api_key": "lm-studio",
        }
    ],
    "timeout": 120  
}

deepseek_coder_config = {
    "config_list": [
        {
            "model": "deepseek-coder-v2-lite-instruct-mlx",
            "base_url": "http://192.168.1.170:1234/v1",
            "api_key": "deepseek-coder-v2-lite-instruct-mlx",
        }
    ],
    "timeout": 120  
}


deepseek_reasoner_config = {
    "config_list": [
        {
            "model": "deepseek-reasoner",
            "price": [0.00014, 0.00219],  # [prompt_price_per_1k, completion_token_price_per_1k]
            "base_url": "https://api.deepseek.com",
            "api_key": deepseek_api_key,
        }
    ],
    "timeout": 120  
}


# Simple AssistantAgent

In [8]:

task = '''
        Write a concise but engaging blogpost about
       AI Agents. Make sure the blogpost is
       within 200 words.
       '''
writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogposts (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=deepseek_reasoner_config,
)


In [9]:


reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])
print(reply)


**AI Agents: The Silent Revolutionaries in Your Daily Life**  

Imagine a digital assistant that books meetings, adjusts your smart thermostat, and even negotiates bills—all without human input. Meet AI agents: autonomous systems designed to perceive, learn, and act. Unlike traditional software, these agents adapt dynamically, leveraging machine learning and real-time data to make decisions.  

From chatbots resolving customer queries to algorithms optimizing supply chains, AI agents are quietly transforming industries. In healthcare, they analyze patient data to recommend treatments. In finance, they detect fraud faster than any human. Even your Netflix recommendations? That’s an AI agent learning your preferences.  

What makes them revolutionary is their ability to operate independently. They set goals, assess environments, and refine strategies through trial and error. Want a travel plan tailored to your budget and interests? An AI agent can scour flights, hotels, and weather forec

# 2 Agents

In [12]:

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogposts (with title) on given topics. You must polish your "
        "writing based on the feedback you receive from other agents and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=deepseek_reasoner_config,
)

critic = autogen.AssistantAgent(
    name="Critic",
    llm_config=deepseek_coder_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)
chat_result = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)


pprint.pprint(chat_result.summary)

Critic (to Writer):


        Write a concise but engaging blogpost about
       AI Agents. Make sure the blogpost is
       within 200 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**AI Agents: The Silent Revolution Reshaping Our World**  

Imagine a world where your digital assistant doesn’t just follow commands but anticipates your needs, schedules your meetings, and even negotiates deals on your behalf. Welcome to the era of **AI agents**—autonomous systems designed to think, learn, and act independently.  

Unlike traditional software, AI agents leverage machine learning and real-time data to adapt dynamically. From chatbots resolving customer queries to self-driving cars navigating traffic, these agents are infiltrating industries. Healthcare uses them for diagnostics, finance deploys them for fraud detection, and logistics relies on them for optimizing supply chains.  

What makes AI agents revolutionary? Th

# Multi-agent refinement of blogpost

In [13]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO_Reviewer",
    llm_config=openAI_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

legal_reviewer = autogen.AssistantAgent(
    name="Legal_Reviewer",
    llm_config=openAI_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

ethics_reviewer = autogen.AssistantAgent(
    name="Ethics_Reviewer",
    llm_config=llama_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [14]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta_Reviewer",
    llm_config=deepseek_reasoner_config,
    system_message="You are a meta reviewer, you aggregate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

In [15]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

In [16]:
review_chats = [ # This is our nested chat
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": 
        {
        "summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",
        },
     "max_turns": 1},
    
    {
     "recipient": legal_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    
    {"recipient": ethics_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
    
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
      "max_turns": 1},
]

In [17]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [18]:
chat_results = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       AI Agents. Make sure the blogpost is
       within 200 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**AI Agents: The Silent Revolution Reshaping Our World**  

Imagine a world where your digital assistant doesn’t just follow commands but anticipates your needs, schedules your meetings, and even negotiates deals on your behalf. Welcome to the era of **AI agents**—autonomous systems designed to think, learn, and act independently.  

Unlike traditional software, AI agents leverage machine learning and real-time data to adapt dynamically. From chatbots resolving customer queries to self-driving cars navigating traffic, these agents are infiltrating industries. Healthcare uses them for diagnostics, finance deploys them for fraud detection, and logistics relies on them for optimizing supply chains.  

What makes AI agents revolutionary? Th